In [ ]:
import pandas as pd


parquet_file_path = '/content/test-00000-of-00001.parquet'
data = pd.read_parquet(parquet_file_path)
csv_file_path = 'output.csv'
data.to_csv(csv_file_path, index=False)
print(f"Файл успешно сохранен как CSV: {csv_file_path}")


Файл успешно сохранен как CSV: output.csv


In [27]:
# Set your API key
api_key= "api_key"


In [ ]:
fd= pd.read_csv('/content/token_prob_predictions.csv')

In [ ]:
!pip install openai==0.28

  Using cached openai-0.28.0-py3-none-any.whl.metadata (13 kB)
Using cached openai-0.28.0-py3-none-any.whl (76 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.75.0
    Uninstalling openai-1.75.0:
      Successfully uninstalled openai-1.75.0


# Log Probs

In [ ]:
import pandas as pd
data=pd.read_csv('/content/output.csv')

In [ ]:
import openai
import numpy as np
from IPython.core.display import display, HTML

MODEL = "gpt-4o"
TOP_LOGPROBS = 3
TEMPERATURE = 0

sentence_list = [
    "The cat sat on the",
    "The stock market is",
    "The weather today is"
]

html_output = ""
high_prob_completions = {}
low_prob_completions = {}

def get_completion(prompt):
    return openai.ChatCompletion.create(
        model=MODEL,
        messages=[{"role": "user", "content": prompt}],
        logprobs=True,
        max_tokens=1,
        temperature=TEMPERATURE,
        top_logprobs=TOP_LOGPROBS
    )

for sentence in sentence_list:
    prompt = f"Complete this sentence with the next word only: {sentence}"
    try:
        response = get_completion(prompt)
        choice = response["choices"][0]
        top_probs = choice.get("logprobs", {}).get("top_logprobs", [])

        html_output += f"<h4>Sentence: <code>{sentence}</code></h4>"

        if top_probs and len(top_probs) > 0:
            for token, logprob in top_probs[0].items():
                prob = np.exp(logprob)
                color = "green" if prob > 0.7 else ("orange" if prob > 0.4 else "red")
                html_output += (
                    f'<p style="color:{color}">'
                    f"Token: <strong>{token}</strong>, "
                    f"LogProb: {logprob:.4f}, "
                    f"Prob: {prob * 100:.2f}%</p>"
                )
            top_token = max(top_probs[0], key=top_probs[0].get)
            if np.exp(top_probs[0][top_token]) > 0.95:
                high_prob_completions[sentence] = top_token
            elif np.exp(top_probs[0][top_token]) < 0.6:
                low_prob_completions[sentence] = top_token
        else:
            html_output += "<p style='color:gray'>logprobs not available (null)</p>"

    except Exception as e:
        html_output += f"<p style='color:red'>Error for sentence '{sentence}': {str(e)}</p>"

display(HTML(html_output))

print("High Probability Completions:", high_prob_completions)
print("Low Probability Completions:", low_prob_completions)


High Probability Completions: {}
Low Probability Completions: {}


In [ ]:
models = openai.Model.list()
for model in models['data']:
    print(model['id'])


gpt-4o-audio-preview-2024-12-17
dall-e-3
text-embedding-3-large
dall-e-2
o4-mini-2025-04-16
gpt-4o-audio-preview-2024-10-01
o4-mini
gpt-4.1-nano
gpt-4.1-nano-2025-04-14
gpt-4o-realtime-preview-2024-10-01
gpt-4o-realtime-preview
babbage-002
tts-1-hd-1106
gpt-4
text-embedding-ada-002
o1-2024-12-17
o1-pro-2025-03-19
o1
tts-1-hd
gpt-4o-mini-audio-preview
o1-pro
gpt-4o-audio-preview
o1-preview-2024-09-12
gpt-4o-mini-realtime-preview
gpt-4.1-mini
gpt-4o-mini-realtime-preview-2024-12-17
gpt-3.5-turbo-instruct-0914
gpt-4o-mini-search-preview
gpt-4.1-mini-2025-04-14
tts-1-1106
chatgpt-4o-latest
davinci-002
gpt-3.5-turbo-1106
gpt-4o-search-preview
gpt-4-turbo
gpt-4o-realtime-preview-2024-12-17
gpt-3.5-turbo-instruct
gpt-3.5-turbo
gpt-4-turbo-preview
gpt-4o-mini-search-preview-2025-03-11
gpt-4-0125-preview
gpt-4o-2024-11-20
whisper-1
gpt-4o-2024-05-13
gpt-4-turbo-2024-04-09
gpt-3.5-turbo-16k
o1-preview
gpt-4-0613
gpt-4.5-preview
gpt-4.5-preview-2025-02-27
gpt-4o-search-preview-2025-03-11
tts-1
om

In [ ]:
import openai
import numpy as np

# Try with experimental logprobs support
try:
    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "The cat sat on the"}
        ],
        max_tokens=1,
        temperature=0,
        logprobs=True,  # Experimental parameter
        top_logprobs=5   # Experimental parameter
    )

    # full response to inspect structure
    print("Response structure:", response)

    #  ways the logprobs might be structured
    if 'choices' in response and 'logprobs' in response['choices'][0]:
        logprobs = response['choices'][0]['logprobs']
        print("Logprobs found:", logprobs)
    else:
        print("Logprobs not available in the response structure")

except Exception as e:
    print(f"Error: {e}")
    print("The current API version likely doesn't support logprobs for chat models")

Response structure: {
  "id": "chatcmpl-BNgM2KCHxh5kV3TTCmxb96aQAQv7d",
  "object": "chat.completion",
  "created": 1744984054,
  "model": "gpt-4o-2024-08-06",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "mat",
        "refusal": null,
        "annotations": []
      },
      "logprobs": {
        "content": [
          {
            "token": "mat",
            "logprob": -0.1290980577468872,
            "bytes": [
              109,
              97,
              116
            ],
            "top_logprobs": [
              {
                "token": "mat",
                "logprob": -0.1290980577468872,
                "bytes": [
                  109,
                  97,
                  116
                ]
              },
              {
                "token": "The",
                "logprob": -2.1290979385375977,
                "bytes": [
                  84,
                  104,
                  101
    

# One question

In [ ]:
import openai
import numpy as np


openai.api_key = api_key

# single test question
question = "What is the capital of France?"
options = "Paris London Berlin Madrid"
options_list = options.split(" ")

while len(options_list) < 4:
    options_list.append('')

# Create the prompt
prompt = f"Q: {question}\nA. {options_list[0]}\nB. {options_list[1]}\nC. {options_list[2]}\nD. {options_list[3]}\nThe correct answer is:"

print("Prompt that will be sent to GPT-4o:")
print("-" * 50)
print(prompt)
print("-" * 50)

# Function to get answer probabilities using GPT-4o
def get_answer_probability(prompt):
    try:
        print("Attempting to use ChatCompletion API with GPT-4o...")
        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are answering multiple choice questions. Respond with just the letter of the correct answer (A, B, C, or D)."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=1,
            temperature=0,
            logprobs=True,
            top_logprobs=5
        )

        print("\nFull response structure:")
        print(response)

        # Extract the answer from the response first
        answer = response.choices[0].message.content.strip()

        # Try to extract logprobs if available
        if hasattr(response.choices[0], 'logprobs'):
            print("\nLogprobs found in the response!")
            logprobs = response.choices[0].logprobs
            return {"success": True, "method": "chat_completion", "data": logprobs, "answer": answer, "response": response}
        else:
            print("\nNo logprobs in the response, extracting answer only")
            return {"success": True, "method": "chat_completion_no_logprobs", "data": {"answer": answer}, "answer": answer}

    except Exception as e:
        print(f"\nError with ChatCompletion: {e}")
        print("Falling back to Completion API with text-davinci-003...")

        # Try alternative approach with an older model that supports logprobs
        try:
            response = openai.Completion.create(
                model="text-davinci-003",
                prompt=f"Answer with just A, B, C, or D.\n{prompt}",
                max_tokens=1,
                logprobs=5,
                temperature=0
            )

            print("\nCompletion API response structure:")
            print(response)

            # Extract answer for completion API
            answer = response.choices[0].text.strip()
            logprobs = response.choices[0].logprobs.top_logprobs[0]
            return {"success": True, "method": "completion_fallback", "data": logprobs, "answer": answer}
        except Exception as e2:
            print(f"\nSecond error: {e2}")
            return {"success": False, "method": "failed", "error": str(e2)}

# Process the single question
result = get_answer_probability(prompt)
print("\n" + "="*50)
print("RESULTS:")
print("="*50)
print(f"Success: {result.get('success', False)}")
print(f"Method used: {result.get('method', 'unknown')}")
print(f"Model's answer: {result.get('answer', 'No answer found')}")

if result.get('success', False):
    data = result.get('data', {})

    if result.get('method') == "chat_completion_no_logprobs":
        print(f"Answer: {data.get('answer', 'No answer found')}")
    elif result.get('method') == "chat_completion":
        # Handle the nested structure of chat completion logprobs
        if hasattr(data, 'content') and len(data.content) > 0:
            token_data = data.content[0]
            top_logprobs_list = token_data.top_logprobs

            print("\nToken probabilities (sorted):")
            # Sort by logprob (highest first)
            sorted_logprobs = sorted(top_logprobs_list, key=lambda x: x.logprob, reverse=True)

            for item in sorted_logprobs:
                token = item.token
                logprob = item.logprob
                prob = np.exp(logprob) * 100
                print(f"Token: '{token}', LogProb: {logprob:.4f}, Probability: {prob:.2f}%")
        else:
            print("Could not extract logprobs content from the response.")
    elif result.get('method') == "completion_fallback":
        # Handle the structure of completion API logprobs
        # Convert logprobs to probabilities and sort
        probs = {}
        for token, logprob in data.items():
            if isinstance(logprob, (int, float)):
                probs[token] = np.exp(logprob) * 100

        # Sort by probability (descending)
        sorted_probs = sorted(probs.items(), key=lambda x: x[1], reverse=True)

        print("\nToken probabilities (sorted):")
        for token, prob in sorted_probs:
            print(f"Token: '{token}', Probability: {prob:.2f}%")
    else:
        print("Data format not recognized:")
        print(data)
else:
    print(f"Error: {result.get('error', 'Unknown error')}")

Prompt that will be sent to GPT-4o:
--------------------------------------------------
Q: What is the capital of France?
A. Paris
B. London
C. Berlin
D. Madrid
The correct answer is:
--------------------------------------------------
Attempting to use ChatCompletion API with GPT-4o...

Full response structure:
{
  "id": "chatcmpl-BNgYzwvkF9O5HyGLvGG6TWEZzduVN",
  "object": "chat.completion",
  "created": 1744984857,
  "model": "gpt-4o-2024-08-06",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "A",
        "refusal": null,
        "annotations": []
      },
      "logprobs": {
        "content": [
          {
            "token": "A",
            "logprob": 0.0,
            "bytes": [
              65
            ],
            "top_logprobs": [
              {
                "token": "A",
                "logprob": 0.0,
                "bytes": [
                  65
                ]
              },
              {
        

# Final Evoluation

In [31]:
import openai
import json
import time
import os
import numpy as np
import random
from tqdm import tqdm
import subprocess
import re
from collections import defaultdict
import pandas as pd
import csv

INPUT_FILE = "/content/output.csv"
OUTPUT_FILE = "results.json"
SAMPLE_SIZE = None
EVAL_FILE = "eval_results.json"
VIDEO_DURATION_TYPE = "short,medium,long"
RUN_EVALUATION = True
EVALUATION_SCRIPT = "/content/eval_your_results-2.py"

# API key
openai.api_key = api_key

# Constants for token analysis
ANSWER_TOKENS = ['A', 'B', 'C', 'D']
EXTENDED_ANSWER_TOKENS = ['A', 'B', 'C', 'D', ' A', ' B', ' C', ' D', 'a', 'b', 'c', 'd']  # Extended set

def format_question_prompt(question, options):
    """Format a question and its options into a prompt for the model."""
    if isinstance(options, str):
        options_list = options.replace('\n', ' ').split("  ")
    else:
        options_list = options

    while len(options_list) < 4:
        options_list.append('')

    prompt = f"Q: {question}\nA. {options_list[0]}\nB. {options_list[1]}\nC. {options_list[2]}\nD. {options_list[3]}\nThe correct answer is:"
    return prompt

def extract_letter_answer(response, logprobs_data=None):
    """
    Extract the letter answer (A, B, C, D) from a response.

    Args:
        response: The text response from the model
        logprobs_data: Optional dictionary of logprobs to help determine the answer

    Returns:
        The extracted letter (A, B, C, or D) or empty string if not found
    """
    response = response.strip()
    matches = re.search(r'[ABCD]', response)
    if matches:
        return matches[0]

    # If no letter found in response and we have logprobs, use those
    if logprobs_data:
        # highest probability among answer tokens
        max_prob = -float('inf')
        best_token = ""

        for token, data in logprobs_data.items():
            # Check if this token is one of our answer tokens or starts with one
            clean_token = token.strip()
            if clean_token in EXTENDED_ANSWER_TOKENS:
                # Get the base letter (A, B, C, D) from the token
                base_letter = clean_token[0].upper()
                if base_letter in ANSWER_TOKENS:
                    if data["probability"] > max_prob:
                        max_prob = data["probability"]
                        best_token = base_letter

        if best_token:
            return best_token

    # If we still can't find an answer, return empty string
    return ""

def get_answer_probability(prompt, max_retries=3, retry_delay=5):
    """Get answer probabilities from GPT-4o with retries for API errors."""
    for attempt in range(max_retries):
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "You are answering multiple choice questions. Respond with just the letter of the correct answer (A, B, C, or D)."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=1,
                temperature=0,
                logprobs=True,
                top_logprobs=5
            )

            # Extract the answer from the response
            raw_answer = response.choices[0].message.content.strip()

            # Extract logprobs if available
            logprobs_data = {}
            if hasattr(response.choices[0], 'logprobs'):
                logprobs = response.choices[0].logprobs

                # Process logprobs
                if hasattr(logprobs, 'content') and len(logprobs.content) > 0:
                    token_data = logprobs.content[0]
                    top_logprobs_list = token_data.top_logprobs

                    for item in top_logprobs_list:
                        token = item.token
                        logprob = item.logprob
                        prob = np.exp(logprob) * 100
                        logprobs_data[token] = {"logprob": logprob, "probability": prob}

            # Extract the letter answer using both the response and logprobs
            letter_answer = extract_letter_answer(raw_answer, logprobs_data)

            return {
                "success": True,
                "raw_answer": raw_answer,
                "letter_answer": letter_answer,
                "logprobs": logprobs_data
            }

        except Exception as e:
            print(f"Attempt {attempt+1}/{max_retries} failed: {e}")
            if attempt < max_retries - 1:
                sleep_time = retry_delay * (2 ** attempt)
                print(f"Retrying in {sleep_time} seconds...")
                time.sleep(sleep_time)
            else:
                return {
                    "success": False,
                    "error": str(e)
                }

def process_csv_dataset(input_file, output_file, sample_size=None):
    """
    Process a dataset from CSV file using GPT-4o.

    Args:
        input_file: Path to the input CSV file
        output_file: Path to save the results
        sample_size: If specified, process only a random sample of this size
    """
    # Check if the file exists
    if not os.path.exists(input_file):
        raise FileNotFoundError(f"Input file not found: {input_file}")

    # CSV file
    print(f"Loading CSV file: {input_file}")
    try:
        df = pd.read_csv(input_file)
        print("Successfully loaded CSV with pandas")
    except Exception as e:
        print(f"Error loading with pandas: {e}")
        print("Trying alternative CSV loading method...")
        # Alternative loading with csv module
        with open(input_file, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            headers = next(reader)
            rows = list(reader)
        df = pd.DataFrame(rows, columns=headers)
        print("Successfully loaded CSV with csv module")

    print("CSV file structure (first 2 rows):")
    print(df.head(2))
    print("CSV columns:", df.columns.tolist())

    # Map column names to expected names if needed
    column_mapping = {}

    question_columns = ['question', 'Question', 'questions', 'Questions', 'question_text', 'QuestionText']
    for col in question_columns:
        if col in df.columns:
            column_mapping['question'] = col
            break

    options_columns = ['options', 'Options', 'option', 'Option', 'choices', 'Choices']
    for col in options_columns:
        if col in df.columns:
            column_mapping['options'] = col
            break

    answer_columns = ['answer', 'Answer', 'correct_answer', 'CorrectAnswer', 'correct']
    for col in answer_columns:
        if col in df.columns:
            column_mapping['answer'] = col
            break

    # If we couldn't find these columns, try to infer them
    if 'question' not in column_mapping and len(df.columns) >= 1:
        column_mapping['question'] = df.columns[0]
        print(f"Using column '{df.columns[0]}' for questions")

    if 'options' not in column_mapping and len(df.columns) >= 2:
        column_mapping['options'] = df.columns[1]
        print(f"Using column '{df.columns[1]}' for options")

    if 'answer' not in column_mapping and len(df.columns) >= 3:
        column_mapping['answer'] = df.columns[2]
        print(f"Using column '{df.columns[2]}' for answers")

    # Check if we have the minimum required columns
    if 'question' not in column_mapping or 'options' not in column_mapping:
        raise ValueError("Could not identify question and options columns in the CSV")

    if column_mapping:
        inverse_mapping = {v: k for k, v in column_mapping.items()}
        df = df.rename(columns=inverse_mapping)
        print("Renamed columns:", inverse_mapping)

    if sample_size is not None and sample_size < len(df):
        print(f"Taking a random sample of {sample_size} items from {len(df)} total items")
        df = df.sample(n=sample_size, random_state=42)

    # Create a JSON-compatible structure for evaluation script
    data = []

    # group questions (by video_id if available)
    if 'video_id' in df.columns:
        grouped = df.groupby('video_id')
    else:
        num_videos = max(1, len(df) // 20)
        video_ids = [f"video_{i}" for i in range(num_videos)]
        df['video_id'] = [video_ids[i % num_videos] for i in range(len(df))]
        grouped = df.groupby('video_id')

    # Process each group
    for video_id, group_df in grouped:
        if 'domain' in group_df.columns:
            domain = group_df['domain'].iloc[0]
        else:
            domain = "Knowledge"

        if 'sub_category' in group_df.columns:
            sub_category = group_df['sub_category'].iloc[0]
        else:
            sub_category = "Literature & Art"

        if 'duration' in group_df.columns:
            duration = group_df['duration'].iloc[0]
        else:
            duration = "medium"  # Default duration

        valid_task_types = [
            "Temporal Perception", "Spatial Perception", "Attribute Perception",
            "Action Recognition", "Object Recognition", "OCR Problems",
            "Counting Problem", "Temporal Reasoning", "Spatial Reasoning",
            "Action Reasoning", "Object Reasoning", "Information Synopsis"
        ]

        video_item = {
            "video_id": video_id,
            "domain": domain,
            "sub_category": sub_category,
            "duration": duration,
            "missing": False,
            "questions": []
        }

        for _, row in group_df.iterrows():
            question_text = row['question']
            options = row['options']

            # if answer is available
            if 'answer' in row:
                answer = row['answer']
                if answer and not re.match(r'^[ABCD]$', str(answer).strip()):
                    matches = re.search(r'[ABCD]', str(answer))
                    if matches:
                        answer = matches[0]
                    else:
                        answer = "A"
            else:
                answer = "A"

            # Check if task_type is available
            if 'task_type' in row:
                task_type = row['task_type']
                # Ensure task_type is valid
                if task_type not in valid_task_types:
                    task_type = valid_task_types[0]
            else:
                task_type = valid_task_types[0]  # Default task type

            question_item = {
                "question": question_text,
                "options": options,
                "answer": answer,
                "task_type": task_type
            }

            video_item["questions"].append(question_item)

        data.append(video_item)

    print(f"Processing {len(data)} videos with GPT-4o")

    processed_count = 0
    success_count = 0
    total_questions = sum(len(item.get("questions", [])) for item in data)

    for i, item in enumerate(data):
        video_id = item.get("video_id", f"video_{i}")
        print(f"Processing video {i+1}/{len(data)}: {video_id}")

        # Process questions for this item
        questions = item.get("questions", [])

        for j, question in enumerate(tqdm(questions, desc=f"Questions for {video_id}")):
            if "response" in question:
                print(f"Skipping already processed question: {j+1}")
                continue

            prompt = format_question_prompt(question["question"], question["options"])
            # the answer from GPT-4o
            result = get_answer_probability(prompt)

            if result["success"]:
                question["response"] = result["raw_answer"]
                question["extracted_answer"] = result["letter_answer"]
                question["logprobs"] = result["logprobs"]

                processed_count += 1
                success_count += 1
            else:
                question["response"] = "Error: " + result.get("error", "Unknown error")
                question["extracted_answer"] = ""
                question["logprobs"] = {}

                processed_count += 1

            # intermediate results after every 10 questions to avoid losing progress
            if processed_count % 10 == 0 or processed_count == total_questions:
                with open(output_file, 'w') as f:
                    json.dump(data, f, indent=2)
                print(f"Progress saved: {processed_count}/{total_questions} questions processed")

            time.sleep(0.5)

    with open(output_file, 'w') as f:
        json.dump(data, f, indent=2)

    print(f"Processing complete. Results saved to {output_file}")
    print(f"Processed {processed_count}/{total_questions} questions, {success_count} successful")

    return data

def prepare_for_evaluation(data, eval_file):

    eval_data = []

    for item in data:
        eval_item = item.copy()
        eval_questions = []

        for question in item.get("questions", []):
            eval_question = question.copy()

            # the expected fields are present
            if "extracted_answer" in eval_question and not "response" in eval_question:
                eval_question["response"] = eval_question["extracted_answer"]
            elif "response" in eval_question and not eval_question["response"]:
                # If response is empty but we have an extracted answer
                if "extracted_answer" in eval_question and eval_question["extracted_answer"]:
                    eval_question["response"] = eval_question["extracted_answer"]

            # response contains just the letter if possible
            if "response" in eval_question:
                response = eval_question["response"]
                matches = re.search(r'[ABCD]', response)
                if matches:
                    letter = matches[0]
                    eval_question["response"] = f"The answer is {letter}. {response}"


            if "logprobs" in eval_question:
                del eval_question["logprobs"]
            if "extracted_answer" in eval_question:
                del eval_question["extracted_answer"]

            eval_questions.append(eval_question)

        eval_item["questions"] = eval_questions
        eval_data.append(eval_item)

    # evaluation-ready data
    with open(eval_file, 'w') as f:
        json.dump(eval_data, f, indent=2)

    print(f"Evaluation-ready data saved to {eval_file}")
    return eval_data

def run_evaluation(results_file, video_duration_type, eval_script):
    """Run the evaluation script on the results."""
    if not os.path.exists(eval_script):
        print(f"Warning: Evaluation script not found at {eval_script}")
        print("Please verify the path to eval_your_results-2.py")
        return False

    cmd = [
        "python", eval_script,
        "--results_file", results_file,
        "--video_duration_type", video_duration_type,
        "--return_categories_accuracy",
        "--return_task_types_accuracy"
    ]

    print(f"Running evaluation command: {' '.join(cmd)}")
    result = subprocess.run(cmd, capture_output=True, text=True)

    print("Evaluation Output:")
    print(result.stdout)

    if result.stderr:
        print("Evaluation Errors:")
        print(result.stderr)

    eval_output_file = f"evaluation_results_{video_duration_type.replace(',', '_')}.txt"
    with open(eval_output_file, 'w') as f:
        f.write(result.stdout)

    print(f"Evaluation results saved to {eval_output_file}")
    return True

def analyze_results(results_file):
    """Analyze the processed results for insights."""
    with open(results_file, 'r') as f:
        data = json.load(f)

    # statistics
    stats = {
        "total_questions": 0,
        "answered_questions": 0,
        "high_confidence_answers": 0,  # When top token has >80% probability
        "answers_by_letter": defaultdict(int),
        "confidence_distribution": defaultdict(int),
    }

    for item in data:
        for question in item.get("questions", []):
            stats["total_questions"] += 1
            if "extracted_answer" in question and question["extracted_answer"]:
                extracted_answer = question["extracted_answer"]
            elif "response" in question:
                matches = re.search(r'[ABCD]', question["response"])
                extracted_answer = matches[0] if matches else ""
            else:
                extracted_answer = ""

            if extracted_answer:
                stats["answered_questions"] += 1
                stats["answers_by_letter"][extracted_answer] += 1


            logprobs = question.get("logprobs", {})
            max_prob = 0

            if isinstance(logprobs, dict):
                for token, data in logprobs.items():
                    if isinstance(data, dict) and "probability" in data:
                        clean_token = token.strip()
                        if clean_token in EXTENDED_ANSWER_TOKENS:
                            prob = data.get("probability", 0)
                            if prob > max_prob:
                                max_prob = prob

                # Categorize confidence
                if max_prob > 0:
                    confidence_bracket = int((max_prob // 10) * 10)  # Round down to nearest 10
                    stats["confidence_distribution"][confidence_bracket] += 1

                    if max_prob > 80:
                        stats["high_confidence_answers"] += 1


    print("\n===== RESULTS ANALYSIS =====")
    print(f"Total questions: {stats['total_questions']}")

    if stats['total_questions'] > 0:
        answered_pct = stats['answered_questions']/stats['total_questions']*100
    else:
        answered_pct = 0
    print(f"Questions with extracted answers: {stats['answered_questions']} ({answered_pct:.2f}%)")

    if stats['total_questions'] > 0:
        high_conf_pct = stats['high_confidence_answers']/stats['total_questions']*100
    else:
        high_conf_pct = 0
    print(f"High confidence answers (>80%): {stats['high_confidence_answers']} ({high_conf_pct:.2f}%)")

    print("\nAnswer distribution by letter:")
    if stats['answered_questions'] > 0:
        for letter, count in sorted(stats["answers_by_letter"].items()):
            print(f"  {letter}: {count} ({count/stats['answered_questions']*100:.2f}%)")
    else:
        print("  No answered questions to analyze")

    print("\nConfidence distribution:")
    if stats['total_questions'] > 0:
        for bracket, count in sorted(stats["confidence_distribution"].items()):
            print(f"  {bracket}-{bracket+10}%: {count} ({count/stats['total_questions']*100:.2f}%)")
    else:
        print("  No questions with confidence data")

    return stats

# Main execution
if __name__ == "__main__":
    print(f"Starting dataset processing with the following configuration:")
    print(f"- Input file: {INPUT_FILE}")
    print(f"- Output file: {OUTPUT_FILE}")
    print(f"- Evaluation file: {EVAL_FILE}")
    print(f"- Video duration type: {VIDEO_DURATION_TYPE}")
    print(f"- Sample size: {SAMPLE_SIZE if SAMPLE_SIZE else 'All data'}")
    print(f"- Run evaluation: {RUN_EVALUATION}")
    print(f"- Evaluation script: {EVALUATION_SCRIPT}")

    data = process_csv_dataset(INPUT_FILE, OUTPUT_FILE, SAMPLE_SIZE)
    eval_data = prepare_for_evaluation(data, EVAL_FILE)
    analyze_results(OUTPUT_FILE)

    # evaluation
    if RUN_EVALUATION:
        run_evaluation(EVAL_FILE, VIDEO_DURATION_TYPE, EVALUATION_SCRIPT)

Starting dataset processing with the following configuration:
- Input file: /content/output.csv
- Output file: results.json
- Evaluation file: eval_results.json
- Video duration type: short,medium,long
- Sample size: All data
- Run evaluation: True
- Evaluation script: /content/eval_your_results-2.py
Loading CSV file: /content/output.csv
Successfully loaded CSV with pandas
CSV file structure (first 2 rows):
   video_id duration     domain        sub_category  \
0         1    short  Knowledge  Humanity & History   
1         1    short  Knowledge  Humanity & History   

                                           url      videoID question_id  \
0  https://www.youtube.com/watch?v=fFjv93ACGo8  fFjv93ACGo8       001-1   
1  https://www.youtube.com/watch?v=fFjv93ACGo8  fFjv93ACGo8       001-2   

              task_type                                           question  \
0      Counting Problem  When demonstrating the Germany modern Christma...   
1  Information Synopsis                  

Questions for 1: 100%|██████████| 3/3 [00:03<00:00,  1.21s/it]


Processing video 2/900: 2


Questions for 2: 100%|██████████| 3/3 [00:02<00:00,  1.01it/s]


Processing video 3/900: 3


...

Processing video 900/900: 900


Questions for 900:  67%|██████▋   | 2/3 [00:01<00:00,  1.07it/s]

Progress saved: 2700/2700 questions processed


Questions for 900: 100%|██████████| 3/3 [00:03<00:00,  1.04s/it]


Processing complete. Results saved to results.json
Processed 2700/2700 questions, 2700 successful
Evaluation-ready data saved to eval_results.json

===== RESULTS ANALYSIS =====
Total questions: 2700
Questions with extracted answers: 2700 (100.00%)
High confidence answers (>80%): 1261 (46.70%)

Answer distribution by letter:
  A: 987 (36.56%)
  B: 643 (23.81%)
  C: 650 (24.07%)
  D: 420 (15.56%)

Confidence distribution:
  20-30%: 6 (0.22%)
  30-40%: 146 (5.41%)
  40-50%: 332 (12.30%)
  50-60%: 358 (13.26%)
  60-70%: 317 (11.74%)
  70-80%: 280 (10.37%)
  80-90%: 298 (11.04%)
  90-100%: 957 (35.44%)
  100-110%: 6 (0.22%)
Running evaluation command: python /content/eval_your_results-2.py --results_file eval_results.json --video_duration_type short,medium,long --return_categories_accuracy --return_task_types_accuracy
Evaluation Output:
Evaluation on video Type: short
-------------------------------------
Video Categories
-------------------------------------
Knowledge:  46.3%
Film & Televi